In [ ]:
import SimpleITK as sitk
import numpy as np

# --- Load CT ---
img = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan.nii", 
                     sitk.sitkFloat32)

# --- Smooth slightly to reduce noise (optional) ---
s_img = sitk.CurvatureFlow(img, timeStep=0.125, numberOfIterations=3)

# --- CT head threshold ---
# Keep everything above -300 HU (keeps skin, soft tissue, bone; removes air and most couch)
mask = sitk.BinaryThreshold(s_img, 
                            lowerThreshold=-300, 
                            upperThreshold=3000, 
                            insideValue=1, 
                            outsideValue=0)

# --- Fill holes ---
mask_filled = sitk.BinaryFillhole(mask, fullyConnected=True)

# --- Largest connected component = head ---
cc = sitk.ConnectedComponent(mask_filled)
stats = sitk.LabelShapeStatisticsImageFilter()
stats.Execute(cc)
largest_label = max(stats.GetLabels(), key=lambda l: stats.GetPhysicalSize(l))

final_mask = sitk.BinaryThreshold(cc, 
                                  lowerThreshold=largest_label, 
                                  upperThreshold=largest_label, 
                                  insideValue=1, 
                                  outsideValue=0)

# Save mask
final_mask_8bit = sitk.Cast(final_mask, sitk.sitkUInt8)
sitk.WriteImage(final_mask_8bit, 
                r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mask.nii")
print("Saved mask successfully")
